In [24]:
import datetime
import requests
from bs4 import BeautifulSoup

# PANDASCORE API KEY - GET ONE HERE : https://api.pandascore.co/
API_KEY = 'YOUR_API_KEY

#------------------ TOURNAMENTS IDS -----------------------#
'''
LPL GROUP A : 1402
LPL GROUP B : 1401
LPL GROUP AB : 1400
LCK : 1399
NACS : 1421
TR : 1420
EULCS : 1398
NALCS : 1205
from https://api.pandascore.co/lol/tournaments/running.json
'''
#regions = '1205,1398,1399,1400,1401,1402,1420,1421'
regions = '1490' # working on EU LCS Playoffs for bo5 stats

#-------------------- UTILS --------------------------------#

def get_opponents(t1, t2):
    return get_team_name(t1) + '-vs-' + get_team_name(t2)

def get_team_name(t):
    name = t["opponent"]["name"]
    name = format_lolesports(name)
    name = name.replace(" ", "-")
    return name.lower()

def format_lolesports(s):
    if s == 'FC Schalke 04 Esports':
        return 'FC Schalke 04'
    return s

def print_match_links(links):
    for l in links:
        if "Voir les stats" in l:
            print(l.get('href'))

def request_lolesports(match_name):
    url = 'https://eu.lolesports.com/fr/eu-lcs/eu_2018_summer/match/2018-08-31/'+match_name
    return requests.get(url=url)

def parse_match_page(resp):
    soup = BeautifulSoup(resp.text, 'html.parser')
    page_links = soup.find_all('a')
    print_match_links(page_links)


#------------------ GET YESTERDAY MATCHES ------------------#
today = datetime.date.today( )
yesterday = today - datetime.timedelta(days=1)

params = {
    'token': API_KEY,
    'range[begin_at]' : str(yesterday)+','+str(today),
    'filter[tournament_id]' : regions
}

url = 'https://api.pandascore.co/lol/matches.json'
resp = requests.get(url=url, params=params)

matches = resp.json()

#-------------------- GET STATS FROM LOLESPORTS ------------#

#def request_lolesports(url):

for match in matches:
    # get opponents
    match_name = get_opponents(match["opponents"][0], match["opponents"][1])
    resp = request_lolesports(match_name)    
    if resp.status_code == requests.codes.ok : 
        parse_match_page(resp)

    else: # Team names need to be in the right order
        match_name = get_opponents(match["opponents"][1], match["opponents"][0])
        resp = request_lolesports(match_name)
        if resp.status_code == requests.codes.ok :
            parse_match_page(resp)


http://matchhistory.euw.leagueoflegends.com/fr/#match-details/TRLH3/1002670054?gameHash=29dc83b15284e15d
http://matchhistory.euw.leagueoflegends.com/fr/#match-details/TRLH3/1002670067?gameHash=4a981a62396da565
http://matchhistory.euw.leagueoflegends.com/fr/#match-details/TRLH3/1002670068?gameHash=866a44676140006d
http://matchhistory.euw.leagueoflegends.com/fr/#match-details/TRLH3/1002670069?gameHash=c15fda85c6b9fe48
